In [ ]:
from flytekit.configuration import set_flyte_config_file, platform
set_flyte_config_file("../../notebook.config")

print("Connected to {}".format(platform.URL.get()))

def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))

# Write a workflow with shared tasks and workflows

Flyte tasks and workflows are natively shareable, i.e., like an library you can introduce a dependency on a remote task/workflow in flyte. Flytekit does this using the *fetch* method.

*To get started, lets first determine the version of the tasks and workflows we will use. In cookbook this corresponds to the last successfully built git-sha*

In [ ]:
version="f6268cf16c52bb457b3a3404e98e68b307bad834"

## Lets fetch the *scale* task

In [ ]:
from flytekit.common.tasks.task import SdkTask
scale_task = SdkTask.fetch("flytesnacks", "development", "recipes.task.tasks.scale", version=version)

## Instead of the *rotate* task we will fetch the *Backfill* workflow which is nothing but a wrapper around the rotate task

This shows that workflows can be shared as well. WE will also show how to create a composite workflow with a task and another subworkflow

In [ ]:
from flytekit.common.launch_plan import SdkLaunchPlan
shared_backfill_lp = SdkLaunchPlan.fetch("flytesnacks", "development", "recipes.interaction.backfill.BackfillWorkflow", version=version)

## Let us create a workflow with these *fetched* tasks and workflows

In [ ]:
from flytekit.sdk.types import Types
from flytekit.sdk.workflow import workflow_class, Input, Output, workflow

@workflow_class
class WorkflowFromSharedTasks():
    in_image = Input(Types.Blob, default=Types.Blob.create_at_known_location(
        "https://miro.medium.com/max/1400/1*qL8UYfaStcEo_YVPrA4cbA.png"))
    angle = Input(Types.Float, default=180.0)
    scale = Input(Types.Integer, default=2)

    scaled = scale_task(image=in_image, scale_factor=scale)
    rotated = shared_backfill_lp(in_image=scaled.outputs.out_image, angle=angle)
    
    out_image = Output(rotated.outputs.out_image, sdk_type=Types.Blob)

## Register them

In [ ]:
shared_lp = WorkflowFromSharedTasks.create_launch_plan()

In [ ]:
WorkflowFromSharedTasks.register(project="flytesnacks", domain="development", name="SharedWorkflow", version="1")
shared_lp.register(project="flytesnacks", domain="development", name="SharedWorkflow", version="1")

## Launch and check the outputs

In [ ]:
exe=shared_lp.launch(project="flytesnacks", domain="development")

In [ ]:
print_console_url(exe)

In [ ]:
exe.wait_for_completion()

In [ ]:
exe.outputs

In [ ]:
img = exe.outputs["out_image"]